In [83]:
import pandas as pd
from scipy.spatial import procrustes
from scipy.linalg import orthogonal_procrustes
from scipy.spatial.distance import pdist
from sklearn.manifold import MDS
import numpy as np

In [2]:
similarities = pd.read_csv("chatgpt_similarities.csv")

In [41]:
sims = similarities.pivot(index="Rock1", columns="Rock2", values="ChatGPT Rating").fillna(0).to_numpy()
sims = np.pad(sims, pad_width=((0, 1), (1, 0)), mode='constant', constant_values=0)
sims += np.eye(30, 30) * 9
for i in range(30):
    for j in range(i+1, 30):
        sims[j, i] = sims[i, j]
dissims = 9 - sims

In [44]:
mds = MDS(n_components=8, metric=False, n_init=10, dissimilarity='precomputed', normalized_stress=True)
mds_solution = mds.fit_transform(dissims)

In [46]:
mds.stress_

0.12634040695282794

In [75]:
ratings = pd.read_csv("chatgpt_ratings.csv")
ratings = ratings.pivot(index="Rock", columns="Dimension", values="ChatGPT Rating").to_numpy()
ratings = np.pad(ratings, pad_width=((0, 0), (0, 2)), mode='constant', constant_values=0)


In [87]:
mtx1 = np.array(ratings, dtype=np.float64, copy=True)
mtx2 = np.array(mds_solution, dtype=np.float64, copy=True)

# translate all the data to the origin
mtx1 -= np.mean(mtx1, 0)
mtx2 -= np.mean(mtx2, 0)

norm1 = np.linalg.norm(mtx1)
norm2 = np.linalg.norm(mtx2)

# change scaling of data (in rows) such that trace(mtx*mtx') = 1
mtx1 /= norm1
mtx2 /= norm2

# transform mtx2 to minimize disparity
R, s = orthogonal_procrustes(mtx1, mtx2)
mtx2 = np.dot(mtx2, R.T)

mtx2 *= norm2